In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U flash_attn
!pip install -q -U datasets==3.6.0

#help track which versions of libraries we're using
!pip list | grep transformers
!pip list | grep accelerate
!pip list | grep bitsandbytes
!pip list | grep datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 79.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.8 MB/s eta 0:00:00
sentence-transformers                    5.1.2
transformers                             4.57.1
accelerate                               1.11.0
bitsandbytes                             0.48.2
datasets                                 3.6.0
tensorflow-datasets                      4.9.9
vega-datasets                            0.9.0


In [ ]:
import datasets
from transformers import pipeline, BitsAndBytesConfig
import bitsandbytes as bnb
import torch
import random
import pandas as pd
from tqdm import tqdm

In [ ]:
!pip install -q evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
#let's make longer output readable without horizontal scrolling
from pprint import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from datasets import load_from_disk
balanced_selfMA_ds = load_from_disk("/content/drive/MyDrive/266_project/balanced_selfMA_ds")
balanced_selfMA_ds = balanced_selfMA_ds.rename_column('text', 'cleaned_text')

random.seed(42)
torch.manual_seed(42)

Mounted at /content/drive


# Zero shot classification - BART Large MNLI

In [ ]:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# https://huggingface.co/tasks/zero-shot-classification
# https://huggingface.co/facebook/bart-large-mnli

model_id = "facebook/bart-large-mnli"

classifier = pipeline(
  "zero-shot-classification",
   model=model_id,
   device = 0
  #  model_kwargs={"dtype": torch.bfloat16, "quantization_config": quantization_config},
  #  device_map="auto",
  #  trust_remote_code=True,
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
test_data = balanced_selfMA_ds['test']
results = []
predictions = []
true_labels = []

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
for idx, sample in enumerate(tqdm(test_data)):
    text = sample['cleaned_text']
    true_label = sample['label']

    result = classifier(
        text,
        candidate_labels=["microaggression", "normal speech"],
        hypothesis_template="This text is {}."
    )

    # Get prediction (top label)
    pred_label = 1 if result['labels'][0] == "microaggression" else 0
    confidence = result['scores'][0]

    predictions.append(pred_label)
    true_labels.append(true_label)

    results.append({
        'text': text,
        'true_label': true_label,
        'pred_label': pred_label,
        'confidence': confidence
    })

    if (idx + 1) % 100 == 0:
        print(f"\nProcessed {idx + 1} samples")
        current_acc = accuracy_score(true_labels[:idx+1], predictions[:idx+1])
        print(f"Current accuracy: {current_acc:.4f}")

# Final evaluation
accuracy = accuracy_score(true_labels, predictions)
print(f"\n{'='*60}")
print(f"Zero-Shot Classification Accuracy: {accuracy:.4f}")
print(f"{'='*60}")
print("\nClassification Report:")
print(classification_report(
    true_labels,
    predictions,
    target_names=['Not Microaggression', 'Microaggression']
))

 19%|█▊        | 102/546 [00:11<00:36, 12.07it/s]


Processed 100 samples
Current accuracy: 0.5500


 37%|███▋      | 201/546 [00:21<00:29, 11.71it/s]


Processed 200 samples
Current accuracy: 0.5400


 55%|█████▌    | 301/546 [00:28<00:20, 11.82it/s]

...


Processed 400 samples
Current accuracy: 0.5200


 92%|█████████▏| 502/546 [00:52<00:03, 11.62it/s]


Processed 500 samples
Current accuracy: 0.5240


100%|██████████| 546/546 [00:57<00:00,  9.51it/s]


Zero-Shot Classification Accuracy: 0.5220

Classification Report:
                     precision    recall  f1-score   support

Not Microaggression       0.83      0.05      0.10       273
    Microaggression       0.51      0.99      0.67       273

           accuracy                           0.52       546
          macro avg       0.67      0.52      0.39       546
       weighted avg       0.67      0.52      0.39       546



# Zero shot classification - MoritzLaurer/deberta-v3-large-zeroshot-v2.0

https://huggingface.co/collections/MoritzLaurer/zeroshot-classifiers

In [ ]:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# https://huggingface.co/tasks/zero-shot-classification
# https://huggingface.co/facebook/bart-large-mnli

model_id = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"

classifier = pipeline(
  "zero-shot-classification",
   model=model_id,
   device = 0
  #  model_kwargs={"dtype": torch.bfloat16, "quantization_config": quantization_config},
  #  device_map="auto",
  #  trust_remote_code=True,
)

test_data = balanced_selfMA_ds['test']
results = []
predictions = []
true_labels = []

from sklearn.metrics import accuracy_score, classification_report
for idx, sample in enumerate(tqdm(test_data)):
    text = sample['cleaned_text']
    true_label = sample['label']

    result = classifier(
        text,
        candidate_labels=["microaggression", "normal speech"],
        hypothesis_template="This text is {}."
    )

    # Get prediction (top label)
    pred_label = 1 if result['labels'][0] == "microaggression" else 0
    confidence = result['scores'][0]

    predictions.append(pred_label)
    true_labels.append(true_label)

    results.append({
        'text': text,
        'true_label': true_label,
        'pred_label': pred_label,
        'confidence': confidence
    })

    if (idx + 1) % 100 == 0:
        print(f"\nProcessed {idx + 1} samples")
        current_acc = accuracy_score(true_labels[:idx+1], predictions[:idx+1])
        print(f"Current accuracy: {current_acc:.4f}")

# Final evaluation
accuracy = accuracy_score(true_labels, predictions)
print(f"\n{'='*60}")
print(f"Zero-Shot Classification Accuracy: {accuracy:.4f}")
print(f"{'='*60}")
print("\nClassification Report:")
print(classification_report(
    true_labels,
    predictions,
    target_names=['Not Microaggression', 'Microaggression']
))


Device set to use cuda:0
 18%|█▊        | 101/546 [00:10<00:51,  8.68it/s]


Processed 100 samples
Current accuracy: 0.8300


 37%|███▋      | 202/546 [00:19<00:31, 10.83it/s]


Processed 200 samples
Current accuracy: 0.8400


 55%|█████▌    | 302/546 [00:29<00:20, 11.67it/s]

...


Processed 400 samples
Current accuracy: 0.8025


 92%|█████████▏| 501/546 [00:47<00:05,  8.29it/s]


Processed 500 samples
Current accuracy: 0.8160


100%|██████████| 546/546 [00:52<00:00, 10.45it/s]


Zero-Shot Classification Accuracy: 0.8114

Classification Report:
                     precision    recall  f1-score   support

Not Microaggression       0.93      0.68      0.78       273
    Microaggression       0.75      0.95      0.83       273

           accuracy                           0.81       546
          macro avg       0.84      0.81      0.81       546
       weighted avg       0.84      0.81      0.81       546

